In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyodesys.symbolic import SymbolicSys
%matplotlib inline

3 body system in 3 dimensional space => 9 spatial + 9 momentum degrees of freedom

In [ ]:
def dydt(t, y, params=(), be=None):
    x0, y0, z0, x1, y1, z1, x2, y2, z2, px0, py0, pz0, px1, py1, pz1, px2, py2, pz2 = y
    m0, m1, m2 = params
    r01 = be.sqrt((x0 - x1)**2 + (y0 - y1)**2 + (z0 - z1)**2)
    r02 = be.sqrt((x0 - x2)**2 + (y0 - y2)**2 + (z0 - z2)**2)
    r12 = be.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)
    f01 = m0*m1/r01
    f02 = m0*m2/r02
    f12 = m1*m2/r12
    e01 = ((x1 - x0)/r01, (y1 - y0)/r01, (z1 - z0)/r01)
    e02 = ((x2 - x0)/r02, (y2 - y0)/r02, (z2 - z0)/r02)
    e12 = ((x2 - x1)/r12, (y2 - y1)/r12, (z2 - z1)/r12)
    f0 = [f01*c01 + f02*c02 for c01, c02 in zip(e01, e02)]
    f1 = [-f01*c01 + f12*c12 for c01, c12 in zip(e01, e12)]
    f2 = [-f02*c02 - f12*c12 for c02, c12 in zip(e02, e12)]
    return [
        px0/m0, py0/m0, pz0/m0,
        px1/m1, py1/m1, pz0/m1,
        px2/m2, py2/m2, pz2/m2,
        f0[0], f0[1], f0[2],
        f1[0], f1[1], f1[2],
        f2[0], f2[1], f2[2],
    ]  # pos, momentum

In [ ]:
odesys = SymbolicSys.from_callback(dydt, 18, 3)

In [ ]:
y0  = [0, 0, 0, 0, 0, 1, 0, 1, 0] + [0]*9
res = odesys.integrate(5, y0, [2, 3, 4], integrator='cvode', method='adams', nsteps=5000)

In [ ]:
_ = res.plot()

In [ ]:
fig1 = plt.figure()
ax = fig1.gca(projection='3d')
ax.plot(*res.yout[:, 0:3].T, alpha=1.0, color='r')
ax.plot(*res.yout[:, 3:6].T, alpha=1.0, color='g')
ax.plot(*res.yout[:, 6:9].T, alpha=1.0, color='b')